In [ ]:
from src import api
from src import wiki

url = "https://lolpros.gg/player/goksi"
player_data = api.get_player_data_by_lolpros(url, last_n=20)

In [ ]:
from constants import *

get_champ_name_to_id()

In [3]:
import psycopg2
from psycopg2 import extras
import pandas as pd
from datetime import datetime
from src import api
from src import wiki, ROLE_ICONS

try:
    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(
        host="postgresql.r5.websupport.sk",
        database="brch_db",
        user="brch_admin",
        password="Ps8zV>95Dg",
        connect_timeout=600,
    )

    with conn.cursor() as cur:
        # Fetch teams data from the database
        cur.execute("SELECT id, name, queryname, url, stats FROM team")
        teams = cur.fetchall()
        teams = pd.DataFrame(teams, columns=["id", "name", "queryname", "url", "stats"])

        for _, team in teams.iterrows():
            # Handle team params
            # ----------------------------
            query = team["queryname"]
            team_id = team["id"]
            team_picks, bans, bans_against = wiki.get_team_data_from_lol_wiki(query)
            dt_string = datetime.now().strftime("%m-%d-%Y %H:%M:%S")
            stats = {"picks": team_picks, "bans": bans, "bans_against": bans_against}
            stats = extras.Json(stats)

            # Update team stats
            # ----------------------------
            update_query = """
                UPDATE team
                SET stats = %s, last_update = %s
                WHERE id = %s
            """
            cur.execute(update_query, (stats, dt_string, team_id))

            cur.execute(
                f"SELECT id, name, lolprosurl, alternativeids, role FROM player WHERE teamid = {team_id}"
            )
            team_players = cur.fetchall()
            team_players = pd.DataFrame(
                team_players,
                columns=["id", "name", "lolprosurl", "alternativeids", "role"],
            )
            for _, player in team_players.iterrows():
                url = player["lolprosurl"]  # "https://lolpros.gg/player/goksi"
                player_id = player["id"]
                alts = (
                    [] if player["alternativeids"] is None else player["alternativeids"]
                )
                player_data = api.get_player_data_by_lolpros(url, last_n=20, alts=alts)
                player_data = extras.Json(player_data)
                update_player_query = """
                UPDATE player
                SET accounts = %s, last_update = %s, icon = %s
                WHERE id = %s
                """
                cur.execute(
                    update_player_query,
                    (player_data, dt_string, ROLE_ICONS[player["role"]], player_id),
                )

            # Commit the changes to the database
            conn.commit()

except Exception as e:
    print(e)
finally:
    # Close the cursor and the connection
    cur.close()
    conn.close()

2023-06-14 10:24:32,186 - INFO - Looking for team - esport stuba
2023-06-14 10:24:32,187 - INFO - Query url - https://lol.fandom.com/wiki/Hitpoint_2nd_Division_Challengers/2023_Season/Summer_Season/Match_History
2023-06-14 10:24:32,336 - INFO - esport stuba parsing successful
2023-06-14 10:24:33,793 - INFO - player - siriass | ids found - ['Siriassik', 'Jr Rekkles']
2023-06-14 10:24:34,725 - INFO - Siriassik - found 0 games
2023-06-14 10:24:34,728 - INFO - Siriassik | soloq : MASTER I 354 | flexq : DIAMOND I 1
2023-06-14 10:24:35,597 - INFO - Jr Rekkles - found 0 games
2023-06-14 10:24:35,600 - INFO - Jr Rekkles | soloq : DIAMOND II 75 | flexq : Unranked
2023-06-14 10:24:36,869 - INFO - player - samyaza | ids found - ['Samyaza33']
2023-06-14 10:24:40,857 - INFO - Samyaza33 - found 23 games
2023-06-14 10:24:40,862 - INFO - Samyaza33 | soloq : MASTER I 531 | flexq : Unranked
2023-06-14 10:24:42,417 - INFO - player - dzejno | ids found - ['Džejnø']
2023-06-14 10:24:45,057 - INFO - Džejnø 